In [1]:
# Imports used for writing postgis table
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
#used for reading postgis table 
import psycopg2  # (if it is postgres/postgis)

#user for working with spatial/data
import pandas as pd
import geopandas as gpd
#SETTING up 
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
#connecting to db ~ making connection
con = psycopg2.connect(database="walkability", user="postgres", password="1234",
    host="172.19.118.205",port="5432")


In [76]:
#grabbing layers sidewalks 
sql = "SELECT * FROM compliance.to_all" #change this to traffic_signal_auto generated lines 
sidewalks = gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom')

 
sql = "select * FROM compliance.centreline_intersection"
centerline_inx = gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom')


sql = "select * FROM compliance.centreline"
centerlines = gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom')




In [79]:

#setting up an empty array for records
recs =[]
#filtering out sidewalk segments less than 40m
potential_crosswalks = sidewalks[sidewalks.geom.length < 40]
#grabbing the spatial r-tree index from the geodataframe and making it an easily accessed variable
potential_crosswalks_index = potential_crosswalks.sindex
centerlines_index = centerlines.sindex
sidewalks_index = sidewalks.sindex
#filtering centerlines by type
centerlines_fl = centerlines[~centerlines['feature__1'].isin(["Local","Laneway",'Trail','Hydro Line','Expressway Ramp','Geostatistical line','Walkway','Minor Shoreline (Land locked)','Minor Arterial Ramp'
'Busway','Other Ramp','Minor Railway','Collector Ramp','Ferry Route','Creek/Tributary','Major Railway','River'])]
#grabbing only the sidewalk segments that cross centerlines
for idx,centerline in centerlines_fl.iterrows():
    possible_crosswalk_index = list(potential_crosswalks_index.intersection(centerline.geom.bounds))
    if possible_crosswalk_index: 
        possible_crosswalk = potential_crosswalks.iloc[possible_crosswalk_index]
        precise_crosswalks = possible_crosswalk[possible_crosswalk.crosses(centerline.geom)]
        for i, row in precise_crosswalks.iterrows(): 
            recs.append(row.to_dict())
#creating a geodataframe from the list of records
gdf = gpd.GeoDataFrame(recs,geometry='geom')
#dropping the columns that can't be writtent to the database (python date/time objects - can convert these to string - just havvent)
gdf.drop(['created_da','last_edite'],axis=1).to_file("/media/DATADRIVE/walkability/to_add_signals.shp")


